In [1]:
import re
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import pyodbc
import nltk
from nltk.tag import pos_tag
from googletrans import Translator
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import sparknlp
from rake_nltk import Rake







In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dryne\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-KLLTOUUR\SQLEXPRESS;'
                      'Database=Dashboard;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor2=conn.cursor()

In [4]:
query = "SELECT * FROM AllCandidates"

cursor.execute(query)
data = cursor.fetchall()

columns = [column[0] for column in cursor.description]

df = pd.DataFrame(columns=columns)

for row in data:
    df = pd.concat(
        [df, pd.DataFrame([dict(zip(columns, row))])], ignore_index=True)

df.to_excel("AllCandidates.xlsx",index=False)


In [5]:
# translator = Translator(service_urls=['translate.google.com'])


# # Function to translate text to English
# def translate_text(text):
#     translation = translator.translate(text, dest='en')
#     return translation.text

# # Apply translation to the DataFrame column
# df["About"] = df["About"].apply(translate_text)
# df["Title"] = df["Title"].apply(translate_text)
# df["Experience"]=df["Experience"].apply(translate_text)
# df["Education"]=df["Education"].apply(translate_text)



In [6]:

cols_to_clean = ["About", "Experience", "Education",
                 "LicensesAndCertifications", "Skills"]
cols_to_tokenize = ["Experience", "Education",
                    "LicensesAndCertifications", "Skills"]
list_of_keywords=["about","experience","education","licensesandcertifications",
                  "licenses&certifications","skills","licensesetcertifications"]
list_to_clean_skills=["show","by","highly","skilled","ardhaoui","who","ardhaouiwho","ardhaouihas",
                      "all","endorsement","skills","skill","endorsements","given","an","sep","apr","feb","jul","tunisia",
                      "endorsed","this","aymen","byaymen","1","2","3","4","5","6","7","8","9","neoxam","time","tunisie"]
clean_line_list=["skills:","-","->"]
list_year=["yr","y","yer","yar","yrs"]
list_mois=["mos","ms"]
print(df["Experience"][10])

- Defining KPIs.
- Analysis and creation of KPIs using tableau and tableau prep.
- a fake news detector and gold price predictor using machine learning techniques.
Dec 2022 - Present · 6 mos
Neopolis Development · Full-time
- Implementation of all phases in a dashboard using ReactJS, Django, and Django rest framework.

- Calculating measures.
- Call center dashboard.
- Work on Creative Canvas, BMC, SWOT
Tunisia
Tunis, Tunisia
Skills:Docker Products · Gitlab · Linux · Django · Git · Python (Programming Language)
CodeClause · Internship
Skills:Flask · GitHub · Apprentissage automatique · Développement web · Python (langage de programmation) · HTML
Aug 2022 - Oct 2022 · 3 mos
FIDNESS · Internship
- Diversity and inclusion dashboard.
Skills: Leadership · Gestion du stress · Leadership d’équipe · Entrepreneuriat
Skills: Flask · GitHub · Apprentissage automatique · Développement web · Python (langage de programmation) · HTML
+2
Capture.PNG
Show all 7 experiences
Skills: Docker Products · Git

In [7]:
def contains_stopword(string, stopwords):
    words=string.split()
    for word in stopwords:
        if word in words  :
            return True
    return False

#éliminer la redondance
def remove_redundancy(liste):
    if(len(liste) >= 2):
        i = 0
        while i < len(liste) - 1:
            if liste[i] == liste[i + 1]:
                liste.remove(liste[i + 1])
            else:
                i += 1
    return liste
def remove_keywords(liste):
    if not liste:  # Check if the list is empty
        return liste

    # Removing keywords
    first_word = liste[0]
    liste2 = liste[1:] if first_word.lower() in list_of_keywords else liste
    return liste2
#replacer un mot dans une liste 
def replace_word_in_list(word_list, old_word, new_word):
    for i in range(len(word_list)):
        if word_list[i] == old_word:
            word_list[i] = new_word
            
def clean_line(words):
    cleaned_words = []
   
    
    for word in words:
        if word.lower() not in clean_line_list: 
            cleaned_words.append(word)
    if cleaned_words:
        premier=cleaned_words[0]
        premier=premier.capitalize()
        cleaned_words= cleaned_words[1:]
        cleaned_words.insert(0, premier)
    return cleaned_words

    

# nettoyer les données
def clean_text(text, col):
    
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = re.sub(r'http\S+', '', text)
        text = text.lower()
        text=text.replace(":"," ")
        text=text.replace("+","-")
        text=text.replace("*","-")
        text=text.replace(">","-")
        text = text.strip()
        lines = text.splitlines()  # Split the text into lines to preserve original line breaks
        cleaned_lines = []

        for line in lines:
            words = line.split()
            words = remove_keywords(words)
            words = remove_redundancy(words)

            b = True  # Initialize 'b' here with default value True

            if col == "Skills" or col == "Experience":
                for i in range(len(words)):
                    if contains_stopword(words[i], list_to_clean_skills) and words[i] not in cleaned_lines:
                        b = False
                        
                    if words[i] in list_year:
                        words[i] = 'year'
                    elif words[i].lower() in list_mois:
                        words[i] = 'month'
            
            if b:
                cleaned_words = clean_line(words)
                cleaned_lines.append(" ".join(cleaned_words))  # Join the words with space

        cleaned_text = "\n".join(cleaned_lines)  # Join the lines using newline character
        return cleaned_text

   
 for col in cols_to_clean:
    df[col] = df[col].apply(lambda x: clean_text(x, col))

In [13]:
   
print(clean_text(df["Experience"][0],"Experience"))

-gestion et suivi des plans d'action issues des inspections des autorites competentes internes et externe.
-lancement des bons d'approvisionnement
Controle matiere premiere et produits finis.
-realisation des audits externes ( fournisseurs / prestataires)
-elaboration matrice / grille des profils des fonctions / des competences par categorie / par departement / par niveau de maitrise
Pfa
Assurance qualite
-participation au preparatif des inspections externes
-mise en place de la modalite de gestion des audits internes des la selection des auditeurs internes jusqu'a l'evaluation des auditeurs
-gestion des commandes export des la planification jusqu'a l'expedition
-mise en place de la politique de la gestion du personnel habilitation et formation continue, gestion des performances du personnel
-redaction et verification des procedures et de tout document officiel du travail
-maitrise des appareils danalyses hplc,spectrometrie ir, spectrometrie uv, ccm, dissolutest, potentiometre et karl-